## Pydantic Model for Medical Data Validation:

### BenefitUser class inherits from BaseModel.
Fields like <code> age, medical_conditions, income, and employed </code> are defined with their types and constraints.


### Validators:

<code>check_conditions</code>: Ensures only valid medical conditions are included.

<code>check_income</code>: Ensures income is non-negative.

<code>check_age</code>: Ensures age is at least 18.

<code>check_employment_details</code>: Ensures that if the person is employed, they must provide details about their employment.

### Sample Data:

Provided sample data is used to test the model. If the data is valid, it prints the validated data; otherwise, it catches and prints validation errors.


In [95]:
# Pydantic takes care of the issue of dynamic typing
# Type Hints
# Data Validation
# JSON Serialization
from typing import List, Optional
from pydantic import BaseModel, validator, constr, condecimal, conint

class BenefitUser(BaseModel):
    name: str
    account_id: str
    medical_conditions: List[str]
    income: condecimal(max_digits=10, decimal_places=2)
    age: conint(ge=0)
    employed: bool

    employment_status_details: Optional[str]=None

    @validator('name') # name cannot be empty nor numeric 
    def check_name(cls, v):
        if v.isnumeric() or not v:
            raise TypeError('Name {v} is not a valid name'.format(v=v))
        return v

    @validator('account_id') # must be 8 digits long
    def check_account_id(cls, v):
        if len(v) != 8 or not v.isnumeric():
            raise TypeError('account id {v} is not 8 digits long'.format(v=v))
        return v

    @validator('medical_conditions')
    def check_conditions(cls, v):
        valid_conditions = {'diabetes', 'hypertension', 'asthma', 'heart disease'}
        for condition in v:
            if condition not in valid_conditions:
                raise ValueError(f"Invalid medical condition: {condition}")
        return v

    @validator('income')
    def check_income(cls, v):
        if v < 0:
            raise ValueError("Income must be a non-negative number.")
        return v

    @validator('age')
    def check_age(cls, v):
        if v < 18:
            raise ValueError("Age must be at least 18.")
        return v

    @validator('employment_status_details', always=True)
    def check_employment_details(cls, v, values):
        if values['employed'] and not v:
            raise ValueError("Employment status details are required if employed.")
        return v
        

# Sample data
data = {
    'name': 'Doug Benedict',
    'account_id': '5687431v',
    'age': 45,
    'medical_conditions': ['diabetes'],
    'income': '25000.00',
    'employed': True,
    'employment_status_details': 'Full-time'
}
user = BenefitUser(**data)

ValidationError: 1 validation error for BenefitUser
account_id
  account id 5687431v is not 8 digits long (type=type_error)

In [100]:
# Sample data
data = {
    'name': 'Doug Benedict',
    'account_id': '56874319',
    'age': 45,
    'medical_conditions': ['diabetes'],
    'income': '25000.00',
    'employed': True,
    'employment_status_details': 'Full-time'
}

try:
    user = BenefitUser(**data)
    print("Validated Benefit User:", user)
except ValueError as e:
    print("Validation error:", e)

print('JSON representation: {json}'.format(json=user.json()))

Validated Benefit User: name='Doug Benedict' account_id='56874319' medical_conditions=['diabetes'] income=Decimal('25000.00') age=45 employed=True employment_status_details='Full-time'
JSON representation: {"name": "Doug Benedict", "account_id": "56874319", "medical_conditions": ["diabetes"], "income": 25000.0, "age": 45, "employed": true, "employment_status_details": "Full-time"}


In [102]:
# pydantic classes can also parse json 
BenefitUser.parse_raw(user.json())

BenefitUser(name='Doug Benedict', account_id='56874319', medical_conditions=['diabetes'], income=Decimal('25000.0'), age=45, employed=True, employment_status_details='Full-time')

In [107]:
import pydantic
print(pydantic.__version__)

1.10.12


In [119]:
# Update pydantic and let's rewrite out benefit user example without looking at anything to include 
! pip install --upgrade pydantic

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.2/125.2 kB 1.1 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 423.9/423.9 kB 1.1 MB/s eta 0:00:0000:0100:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 1.6 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: pydantic
    Found existing installation: pydantic 1.10.12
    Uninstalling pydantic-1.10.12:
      Successfully uninstalled pydantic-1.10.12


In [31]:
from pydantic import BaseModel, field_validator, conint, condecimal, constr, EmailStr
from typing import List, Optional
import datetime as dt

class BenefitUser(BaseModel):
    name: str
    age: conint(ge=0)
    email: EmailStr 
    application_date: dt.datetime | None
    conditions: List[str]
    income: condecimal(max_digits=10, decimal_places=2)
    employment_status: bool

    employment_status_details: Optional[str] | None

    @field_validator('name')
    def check_name(cls, v) -> constr:
        if v.isnumeric() or not v:
            raise TypeError('Name {v} is not a valid name'.format(v=v))
        return v

    @field_validator('age')
    def check_age(cls, v) -> conint:
        if v < 18:
            raise ValueError('Must be 18 or older')
        return v

    @field_validator('application_date')
    def check_application_date(cls, v) -> dt.datetime:
        if v > dt.datetime.now():
            raise ValueError('Date {v} is in the future'.format(v=v))
        return v

    @field_validator('conditions')
    def check_conditions(cls, v) -> List[str]:
        acceptable_conditions = {'heart disease', 'diabetes', 'cancer', 'bad brain'}
        if not set(v) & acceptable_conditions:
            raise ValueError('None of {v} are acceptable conditions'.format(v=','.join(v)))
        return v

    @field_validator('income')
    def check_income(cls, v) -> condecimal:
        if v < 0:
            raise ValueError('Negative income is invalid')
        return v

dict_data = {
    'name':'Joe',
    'age':'43',
    'email':'joe@gmail.com',
    'application_date':dt.datetime.now(),
    'conditions':['diabetes'],
    'income':30000.00,
    'employment_status':True,
    'employment_status_details':'Full-Time'
}

try:
    user = BenefitUser(**dict_data)
    print('{user.name} validated as a BenefitUser object'.format(user=user))
except Exception as e:
    print('Exception {e} occurred trying to validate BenefitUser'.format(e=e))

In [41]:
# try it with some bad data
dict_data = {
    'name':'Joe',
    'age':'43',
    'email':'joe@gmail.com',
    'application_date':dt.datetime.now(),
    'conditions':['cancer'],
    'income':30000.00,
    'employment_status':True,
    'employment_status_details':'Full-Time'
}

try:
    user = BenefitUser(**dict_data)
    print('{user.name} validated as a BenefitUser object'.format(user=user))
except Exception as e:
    print('Exception {e} occurred trying to validate BenefitUser'.format(e=e))

Exception 1 validation error for BenefitUser
email
  value is not a valid email address: An email address must have an @-sign. [type=value_error, input_value='gmail.com', input_type=str] occurred trying to validate BenefitUser


In [ ]:
# what if we had to fetch json data from an API and validate our model?
import requests
import json

# note since this code is fake it won't function below
url = 'https://some_data.com/data'
r = requests.get(url) # this probably has a lot of data in a real scenario
my_users = [BenefitUser.parse_raw(element.content) for element in r.content] # creates a BenefitUser for each record in the get content